In [1]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

/home/vco/Projects/pm4py-dcr/notebooks
/home/vco/Projects/pm4py-dcr


In [2]:
import pm4py

In [3]:
from pm4py.objects.dcr import semantics as dcr_semantics
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
from pm4py.algo.discovery.dcr_discover import algorithm as dcr_alg

# File names

In [4]:
event_log_file = '/home/vco/Datasets/Sepsis Cases - Event Log.xes'
dcrxml_file = 'models/sepsis_dcr.xml'
dcr_title = 'Sepsis Cases'

# Mine DCR graph using the DisCoveR miner

In [5]:
event_log = pm4py.read_xes(event_log_file,return_legacy_log_object=True)

/home/vco/.pyenv/versions/3.11.0/envs/pm4py-dcr/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1050/1050 [00:00<00:00, 2265.99it/s]


In [6]:
dcr_graph = dcr_alg.apply(input_log=event_log,variant=dcr_alg.DCR_BASIC,findAdditionalConditions=True) # this will need updating to the new pm4py standards (for now it works)

/home/vco/Projects/pm4py-dcr/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [7]:
for k,v in dcr_graph.items():
    print(k,v)

events {'Return ER', 'LacticAcid', 'Admission IC', 'IV Antibiotics', 'ER Registration', 'Admission NC', 'Release B', 'IV Liquid', 'Leucocytes', 'Release D', 'ER Triage', 'Release C', 'CRP', 'Release E', 'Release A', 'ER Sepsis Triage'}
conditionsFor {'Return ER': {'Release D', 'Release C', 'Admission IC', 'Release E', 'Release A'}, 'LacticAcid': set(), 'Admission IC': {'ER Triage', 'ER Registration'}, 'IV Antibiotics': {'ER Sepsis Triage'}, 'ER Registration': set(), 'Admission NC': {'ER Triage', 'ER Registration'}, 'Release B': {'ER Triage', 'Leucocytes', 'CRP', 'ER Registration', 'ER Sepsis Triage'}, 'IV Liquid': set(), 'Leucocytes': set(), 'Release D': {'Leucocytes', 'CRP', 'Admission NC', 'ER Sepsis Triage'}, 'ER Triage': set(), 'Release C': {'Leucocytes', 'CRP', 'Admission NC', 'ER Sepsis Triage'}, 'CRP': set(), 'Release E': {'Leucocytes', 'CRP', 'Admission NC', 'ER Sepsis Triage'}, 'Release A': {'ER Triage', 'ER Registration'}, 'ER Sepsis Triage': set()}
milestonesFor {'Return ER'

# Write the graph to file

In [10]:
dcr_exporter.apply(dcr_graph=dcr_graph,
                    path=dcrxml_file,
                    variant = dcr_exporter.DCR_XML_FULL,
                    dcr_title=dcr_title)

# Read the dcrxml graph file
Works also with graphs downlaoded from the dcrgraphs.net portal

In [11]:
dcr_graph_loaded = dcr_importer.apply(dcrxml_file)
print(dcr_graph_loaded)

{'events': {'AdmissionNC', 'ERRegistration', 'IVLiquid', 'Leucocytes', 'ReleaseC', 'ReleaseD', 'LacticAcid', 'CRP', 'ERSepsisTriage', 'ReleaseE', 'ReleaseA', 'ReturnER', 'AdmissionIC', 'IVAntibiotics', 'ERTriage', 'ReleaseB'}, 'labels': {'AdmissionNC', 'ERRegistration', 'IVLiquid', 'Leucocytes', 'ReleaseC', 'ReleaseD', 'LacticAcid', 'CRP', 'ERSepsisTriage', 'ReleaseE', 'ReleaseA', 'ReturnER', 'AdmissionIC', 'IVAntibiotics', 'ERTriage', 'ReleaseB'}, 'labelMapping': set(), 'conditionsFor': {'ReturnER': {'ReleaseC', 'ReleaseD', 'ReleaseE', 'ReleaseA', 'AdmissionIC'}, 'AdmissionIC': {'ERRegistration', 'ERTriage'}, 'IVAntibiotics': {'ERSepsisTriage'}, 'AdmissionNC': {'ERRegistration', 'ERTriage'}, 'ReleaseB': {'ERRegistration', 'CRP', 'Leucocytes', 'ERSepsisTriage', 'ERTriage'}, 'ReleaseD': {'Leucocytes', 'CRP', 'ERSepsisTriage', 'AdmissionNC'}, 'ReleaseC': {'Leucocytes', 'CRP', 'ERSepsisTriage', 'AdmissionNC'}, 'ReleaseE': {'Leucocytes', 'CRP', 'ERSepsisTriage', 'AdmissionNC'}, 'ReleaseA':

# Play with the execution semantics (note milestones and no-responses are not supported yet)

In [12]:
dcr = dcr_graph_loaded
print(dcr['events']) # all the events

{'AdmissionNC', 'ERRegistration', 'IVLiquid', 'Leucocytes', 'ReleaseC', 'ReleaseD', 'LacticAcid', 'CRP', 'ERSepsisTriage', 'ReleaseE', 'ReleaseA', 'ReturnER', 'AdmissionIC', 'IVAntibiotics', 'ERTriage', 'ReleaseB'}


In [13]:
print(dcr_semantics.enabled(dcr)) # check enabled events

{'ERRegistration', 'LacticAcid', 'CRP', 'ERTriage', 'Leucocytes', 'ERSepsisTriage', 'IVLiquid'}


In [16]:
event_to_execute = 'ERRegistration' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute, dcr) # here you execute the event
print(dcr_semantics.enabled(dcr)) # now these are the newly enabled events

{'LacticAcid', 'CRP', 'ERTriage', 'Leucocytes', 'ERSepsisTriage', 'IVLiquid'}


In [17]:
event_to_execute = 'ERSepsisTriage' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute, dcr) # here you execute the event
print(dcr_semantics.enabled(dcr)) # now these are the newly enabled events

{'LacticAcid', 'CRP', 'ERTriage', 'Leucocytes', 'IVAntibiotics', 'IVLiquid'}


In [18]:
dcr_semantics.is_accepting(dcr) # this checks the acceptance criteria for the dcr graph

True

In [33]:
dcr['marking']['pending']

set()